In [1]:
import pandas as pd
import speedtest
# from IPython.display import display, clear_output
import PySimpleGUI as sg
import gc
import os
import sys

pd.options.display.max_rows = 200
pd.options.display.precision = 2

bit_to_mb_factor = 10**6

main_dir = str(os.path.normpath(os.getcwd()+os.sep+os.pardir))
images_dir = main_dir+os.sep+"images"+os.sep+"exports"+os.sep
output_dir = main_dir+os.sep+"data"+os.sep+"processed"+os.sep

app_logo_fname = "app_logo.ico"

results = dict()
timestamp_fmt = "%d %B, %Y %H:%M:%S"
cities_rgx = "sacramento$|austin$|mexico$|brasilia$|tallahassee$|washington, dc$|new york$"
cities_rgx += cities_rgx + "manchester$|london$|frankfurt$|paris$|rome$|ankara$|moscow$"
cities_rgx += cities_rgx + "new delhi$|tokyo$|canberra$|jakarta$|bangkok$|seoul$"

test_speed = sg.PopupYesNo("Test your worldwide internet speed?", title="WST",
                           icon=images_dir+os.sep+app_logo_fname)

# def close_all():
#     sg.EasyPrintClose()
#     sys.exit()

if test_speed == "Yes":
    s = speedtest.Speedtest()
    servers_df = pd.DataFrame.from_dict(s.get_servers(), orient="index")
    servers_df= servers_df[0].reset_index(drop=True)
    servers_df = pd.DataFrame(servers_df.to_list()).sort_values("d").set_index("id")  # sort by ping
    # servers_df.head(2)

    server_inds = servers_df["name"][servers_df["name"].str.contains(cities_rgx, case=False)]
    server_inds = server_inds.drop_duplicates().index

    start_time = pd.to_datetime('now')
    print = sg.EasyPrint  # (do_not_reroute_stdout=False)
    print("Welcome to Worldwide Speed Test. \n\nFor this to work, please close"+
          " all internet related applications from all devices using the test"+
          " network. Optionally restart the broadband device and set your"+
          " power profile to high performance."+
          "\n\nINFO: start time - "+start_time.strftime(timestamp_fmt)+" - currently speed testing:")

    for i, server in enumerate(server_inds):
        s = speedtest.Speedtest()

        servers = [server]  # has to be one only
        # If you want to test against a specific server
        # servers = [1234]

        s.get_servers(servers)  # run this to update the internal config of speedtest

        threads = None
        # If you want to use a single threaded test
        # threads = 1

        s.download(threads=threads)  # tests download
        s.upload(threads=threads, pre_allocate=False)  # tests upload
        s.results.share()  # shares results with speedtest

        results[server] = s.results.dict()

#         clear_output(wait=True)
        print(servers_df["name"].loc[server]+", "+servers_df["country"].loc[server])

    results_df = pd.DataFrame(results).T

    server_df = pd.DataFrame()
    client_df = pd.DataFrame()

    for idx, server, client in results_df[["server", "client"]].itertuples():
        server_df = server_df.append(server, ignore_index=True)
        client_df = client_df.append(client, ignore_index=True)

    server_df.index, client_df.index = results_df.index, results_df.index
    results_df = results_df.drop(columns=["server", "client"])
    results_df = pd.concat([results_df, server_df, client_df], axis=1, sort=False)

    del server_df, client_df, results
    gc.collect()

    finish_time = pd.to_datetime('now')
    print("INFO: end time - ", str(finish_time) + ". This took",
          round((finish_time - start_time).total_seconds() / 60, 2), "minutes")

    # results_df = pd.read_csv(output_dir+isp_name_package+".csv")
    print("Finished!\n\nMedian download speed is", 
          round(results_df["download"].median()/bit_to_mb_factor, 2), "mb",
          "\nMedian upload speed is", round(results_df["upload"].median()/bit_to_mb_factor, 2), "mb",
          "\nPing is", round(results_df["ping"].median(), 2), "ms")
else:
    break
#     close_all()
    
# display(results_df.head())

save_results = sg.PopupYesNo("Save results as a csv file?", title="WST", icon=images_dir+os.sep+app_logo_fname)

if save_results == "Yes":
    text_in = sg.PopupGetText('Please enter filename (without extension)',
                              title="WST",
                              icon=images_dir+os.sep+app_logo_fname)
    results_df.to_csv(output_dir+text_in+".csv", index=False)
    break
#     close_all()
else:
    break
#     close_all()

SystemExit: 

D:\Users\gjnet\code_projects\py_speed_test\.venv\lib\site-packages\IPython\core\interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
